# Kiosk Auswertungen

Auswertung am 04.01.2024

## Auswertung Updaten mit 24, PayPal_Bargeld!


In [ ]:
from __future__ import annotations
from typing import List, Dict
import sqlite3
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.parser import parse as dt_parse

con = sqlite3.connect('ffeDataBase.sqlite3')
cur = con.cursor()


def dictfetchall(cursor) -> List[dict]:
    """Return all rows from a cursor as a dict"""
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]


def query(*args, **kwargs) -> pd.DataFrame:
    """"""
    return pd.DataFrame(
        dictfetchall(
            cur.execute(*args, **kwargs)
        )
    )



layout: dict = dict(
    height=600,
    width=800,
    font_size=16,
    yaxis_showline=True,
    yaxis_linewidth=1,
    yaxis_ticks='outside',
    xaxis_showline=True,
    font_family='Segoe UI',
    yaxis_gridcolor='#E0E0E0',
    yaxis_linecolor='#A3A3A3',
    xaxis_linecolor='#A3A3A3',
    plot_bgcolor='white',
    xaxis_color='#777777',
    yaxis_color='#777777',
    legend_font_color='#777777',
    title_font_color='#777777',
    yaxis_showgrid=False,
    xaxis_showgrid=False,
    yaxis_rangemode='tozero',
)

color_order: List[str] = ['#356CA5', '#777777']

year_choices: List[int] = list(range(2017, datetime.now().year + 1))

# Monatliche Verkäufe

Alle Monate von Beginn des Kiosks an.

In [ ]:
monthly_selling: pd.DataFrame = query("""
select
	strftime('%m', "gekauftUm") as month,
	strftime('%Y', "gekauftUm") as year,
	sum(verkaufspreis) as verkauft
from kiosk_gekauft
group by month,year
order by year,month
""")

'''fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.plot(
    [x['month']+'.'+x['year'] for x in monthly_selling],
    [x['verkauft']/100 for x in monthly_selling],
)
plt.show()'''

monthly_selling['verkauft'] = monthly_selling.apply(
    lambda x: x['verkauft'] / 100.0,
    axis=1,
)
monthly_selling['date'] = monthly_selling.apply(
    lambda x: date(int(x['year']), int(x['month']), 1),
    axis=1,
)


_layout = deepcopy(layout)
_layout['yaxis_title'] = 'monatliche Verkäufe in €'

fig = go.Figure(
    data=go.Scatter(
        y=monthly_selling['verkauft'],
        x=monthly_selling['date'],
        line=dict(color=color_order[0]),
	),
    layout=_layout,
)
fig.show()

# Unbezahlte Ware

Darstellung per Rate von Beginn des Kiosks an.

Weitere Dateninhalte, nicht dargestellt: absolute Zahlen der Umsätze und absolute Zahlen unbezahlt an jedem Stichtag, an dem eine Inventur durchgeührt wurde.

In [ ]:
unpaid_details: pd.DataFrame = query("""
select 
    *,
    allesUmsatz - dieb as bezahlt
from (
    select 
        date(stamp) as datum,
        sum(verkaufspreis) / 100.0 as allesUmsatz
    from (
        select 
            *
        from kiosk_gekauft a
        join profil_kioskuser b
          on a.kaeufer_id = b.id
    ) a, (
        select 
            datetime(stamp,'+1 day','-1 second') as stamp
        from (
            select
                date(gekauftUm) as stamp
            from (
                select 
                    *
                from kiosk_gekauft a
                join profil_kioskuser b
                  on a.kaeufer_id = b.id
            )
            where username = 'Dieb'
            group by stamp
        )
    ) b
    where gekauftUm <= stamp
    group by stamp
) a
join (
    select 
        date(stamp) as datum,
        sum(verkaufspreis) / 100.0 as dieb
    from (
        select 
            *
        from (
            select 
                *
            from kiosk_gekauft a
            join profil_kioskuser b
              on a.kaeufer_id = b.id
        )
        where username='Dieb'
    ) a, (
        select 
            datetime(stamp,'+1 day','-1 second') as stamp
        from (
            select
                date(gekauftUm) as stamp
            from (
                select 
                    *
                from kiosk_gekauft a
                join profil_kioskuser b
                  on a.kaeufer_id = b.id
            )
            where username = 'Dieb'
            group by stamp
        )
    ) b
    where gekauftUm <= stamp
    group by stamp
) b
    using(datum)
    
order by datum asc
""")

unpaid_details['date'] = unpaid_details.apply(
    lambda x: dt_parse(x['datum']),
    axis=1,
)

unpaid_details['unpaid_rate'] = unpaid_details.apply(
    lambda x: x['dieb'] / x['allesUmsatz'] * 100,
    axis=1,
)

_layout = deepcopy(layout)
_layout['yaxis_title'] = 'Unbezahlt Rate in %'

fig = go.Figure(
    data=go.Scatter(
        y=unpaid_details['unpaid_rate'],
        x=unpaid_details['date'],
        line=dict(color=color_order[0]),
	),
    layout=_layout,
)
fig.show()

## Hier noch Darstellung für isolierte Jahresbetrachtung

# Nutzer:innen

## Neue Nutzer:innen

In [ ]:
new_users_per_year: pd.DataFrame = query("""
select strftime('%Y', date_joined) as year_joined,
       count(*) as cnt
from profil_kioskuser
group by year_joined
order by year_joined
""")

fig, ax = plt.subplots()
trace = ax.bar(
    new_users_per_year.year_joined,
    new_users_per_year.cnt,
    color=color_order[0],
)
ax.set_ylabel('Anzahl neue Nutzer:innen')
ax.bar_label(trace)
plt.show()

## Aktive Nutzer:innen

Personen, die im jeweiligen Jahr aktiv waren. (Nur Login auf Webseite als Indikator! Man könnte noch Käufe und weitere Interaktionen verwenden.)

**Nur das letzte Jahr ist hier aussagekräftig: User, die in vorherigen Jahren auch aktiv waren, werden nur auf das letzte Jahr der Aktivität gewertet...**

In [ ]:
active_users: pd.DataFrame = query("""
select strftime('%Y', last_login) as year_login,
       count(*) as cnt
from profil_kioskuser
group by year_login
order by year_login
""")

active_users = active_users.loc[~active_users.year_login.isna(), :]

fig, ax = plt.subplots()
trace = ax.bar(
    active_users.year_login,
    active_users.cnt,
    color=color_order[0],
)
ax.set_ylabel('Anzahl aktive Nuter:innen')
ax.bar_label(trace)
plt.show()

## Verdiente Einkäufer:innen

In [ ]:
best_buyers: pd.DataFrame = query("""
select
count(*) cnt,
username,
strftime('%Y', "geliefertUm") as year
from kiosk_gekauft
join profil_kioskuser on einkaeufer_id=id
group by einkaeufer_id, year
order by year, username
""")

In [ ]:
from lets_plot import * 
LetsPlot.setup_html()
ggplot() + \
geom_bar(aes(x="year", y="cnt", color="username", fill="username"), data=best_buyers, sampling="none" if best_buyers.size < 50 else sampling_pick(n=50), stat="identity") + \
ggtitle("Verdiente Einkäufer:innen")  +\
ylab("Anzahl der Einkäufe")

## Präferierte Bezahlmethode

In [ ]:
buying_method: pd.DataFrame = query("""
select
gekauft_per,
strftime('%Y', "gekauftUm") as year_bought,
count(*) cnt
from kiosk_gekauft
group by gekauft_per, year_bought
order by year_bought, gekauft_per
""")

grouped = buying_method.groupby('year_bought')

share_per_year: List[dict] = []
for name, group in grouped:
    if {'slack', 'web'}.issubset(set(group.gekauft_per.values)):
        slack_cnt: int = group.loc[group.gekauft_per == 'slack', 'cnt'].item()
        web_cnt: int = group.loc[group.gekauft_per == 'web', 'cnt'].item()
        share_per_year.append(dict(
            year=name,
            slack_share=slack_cnt / (slack_cnt + web_cnt),
            web_share=web_cnt / (slack_cnt + web_cnt),
        ))

fig, ax = plt.subplots()
trace_slack = ax.bar(
    [x['year'] for x in share_per_year],
    [round(x['slack_share'],2)*100 for x in share_per_year],
    label='Slack',
    color=color_order[0],
)
trace_web = ax.bar(
    [x['year'] for x in share_per_year],
    [round(x['web_share'],2)*100 for x in share_per_year],
    bottom=[round(x['slack_share'],2)*100 for x in share_per_year],
    label='Web',
    color=color_order[1],
)
ax.set_ylabel('Aufteilung der Bezahlmethoden in %')
ax.bar_label(trace_slack, label_type='center')
ax.legend()
plt.show()

# Geldwerte

## Kioskbetrieb

Rechnung zunächst ohne zusätzliches Einkommen v.a. durch Spende durch Kontoauflösung, dann werden zusätzliche Einkünfte mit aufgenommen.

In [ ]:
unpaid: pd.DataFrame = query("""
  select
  strftime('%Y', "gekauftUm") as year,
  sum(verkaufspreis) as unpaid
  from kiosk_gekauft
  where kaeufer_id=4
  group by year
  order by year
""")

unpaid_missed_profits: pd.DataFrame = query("""
select
strftime('%Y', "gekauftUm") as year,
sum((verkaufspreis-einkaufspreis)/2) as unpaid_missed_profits
from kiosk_gekauft
where kaeufer_id=4
group by year
order by year
""")

paid: pd.DataFrame = query("""
select
strftime('%Y', "gekauftUm") as year,
sum(verkaufspreis) as paid
from kiosk_gekauft
where kaeufer_id <> 4
group by year
order by year
""")

money_lost: pd.DataFrame = query("""
with bezahlt as (
select
strftime('%Y',datum) as year,
vonnutzer_id,
sum(betrag) as bezahlt
from kiosk_geldtransaktionen
where vonnutzer_id=2
  and zunutzer_id = 73
group by vonnutzer_id, year
)

, bekommen as (
select
strftime('%Y',datum) as year,
zunutzer_id,
sum(betrag) as bekommen
from kiosk_geldtransaktionen 
where zunutzer_id=2
  and vonnutzer_id=73
group by zunutzer_id, year
)

select
coalesce(bezahlt.year, bekommen.year) as year,
(coalesce(bezahlt,0) - coalesce(bekommen,0)) as money_lost
from bezahlt
left join bekommen
 on bezahlt.vonnutzer_id=bekommen.zunutzer_id and bezahlt.year = bekommen.year
union all
select
coalesce(bezahlt.year, bekommen.year) as year,
(coalesce(bezahlt,0) - coalesce(bekommen,0)) as money_lost
from bekommen
left join bezahlt
 on bezahlt.vonnutzer_id=bekommen.zunutzer_id and bezahlt.year = bekommen.year
where bezahlt.vonnutzer_id is null
""")

worth: pd.DataFrame = query("""
with bezahlt as (
select
strftime('%Y',datum) as year,
vonnutzer_id,
sum(betrag) as bezahlt
from kiosk_geldtransaktionen
where vonnutzer_id=3
group by vonnutzer_id, year
)

, bekommen as (
select
strftime('%Y',datum) as year,
zunutzer_id,
sum(betrag) as bekommen
from kiosk_geldtransaktionen 
where zunutzer_id=3
group by zunutzer_id, year
)

select
bezahlt.year,
-(bezahlt-bekommen) as diff
from bezahlt
left join bekommen
 on bezahlt.vonnutzer_id=bekommen.zunutzer_id and bezahlt.year = bekommen.year
""")
worth['worth'] = worth['diff'].cumsum()

sell_value: pd.DataFrame = query("""
with all_kiosk_items as (
select
 strftime('%Y', "geliefertUm") as year,
 verkaufspreis
from kiosk_kiosk a
left join (
  select
    c.id,
    a.verkaufspreis
  from kiosk_produktverkaufspreise a
  join (
      select
        a.produktpalette_id,
        max(a."gueltigAb") as "gueltigAb"
      from
        kiosk_produktverkaufspreise a
      group by
        a.produktpalette_id
    ) b
    using (produktpalette_id,"gueltigAb")
  join kiosk_produktpalette c
      on a.produktpalette_id = c.id
) b
 on a.produktpalette_id = b.id

union all

select
 strftime('%Y', "geliefertUm") as year,
 verkaufspreis
from kiosk_gekauft
where cast(strftime('%Y', "gekauftUm") as integer) > cast(strftime('%Y', "geliefertUm") as integer)
)

select
year,
sum(verkaufspreis) as sell_value
from all_kiosk_items
group by year
order by year
""")

buy_value: pd.DataFrame = query("""
with all_kiosk_items as (
select
 strftime('%Y', "geliefertUm") as year,
 einkaufspreis
from kiosk_kiosk a

union all

select
 strftime('%Y', "geliefertUm") as year,
 einkaufspreis
from kiosk_gekauft
where cast(strftime('%Y', "gekauftUm") as integer) > cast(strftime('%Y', "geliefertUm") as integer)
)

select
year,
sum(einkaufspreis) as buy_value
from all_kiosk_items
group by year
order by year
""")

accounts_value: pd.DataFrame = query("""
with bezahlt as (
select
strftime('%Y',datum) as year,
vonnutzer_id,
sum(betrag) as bezahlt
from kiosk_geldtransaktionen
group by vonnutzer_id, year
)

, bekommen as (
select
strftime('%Y',datum) as year,
zunutzer_id,
sum(betrag) as bekommen
from kiosk_geldtransaktionen
group by zunutzer_id, year
)

, stand as (
select
coalesce(bezahlt.year, bekommen.year) as year,
coalesce(bekommen.zunutzer_id, bezahlt.vonnutzer_id) as user_id,
-(coalesce(bezahlt,0) - coalesce(bekommen,0)) as diff
from bezahlt
left join bekommen
 on bezahlt.vonnutzer_id=bekommen.zunutzer_id and bezahlt.year = bekommen.year
union all
select
coalesce(bezahlt.year, bekommen.year) as year,
coalesce(bekommen.zunutzer_id, bezahlt.vonnutzer_id) as user_id,
-(coalesce(bezahlt,0) - coalesce(bekommen,0)) as diff
from bekommen
left join bezahlt
 on bezahlt.vonnutzer_id=bekommen.zunutzer_id and bezahlt.year = bekommen.year
where bezahlt.vonnutzer_id is null
)

select
  year,
  sum(diff) as value
from stand
where user_id not in (2,3,4,72,73,102)
group by year
order by year
""")
accounts_value['accounts_value'] = accounts_value['value'].cumsum()


buy_value_sold_items: pd.DataFrame = query("""
  select
  strftime('%Y', "gekauftUm") as year,
    sum(einkaufspreis) as buy_value_sold_items
  from kiosk_gekauft
  group by year
  order by year
""")

sell_value_sold_items: pd.DataFrame = query("""
  select
  strftime('%Y', "gekauftUm") as year,
    sum(verkaufspreis) as sell_value_sold_items
  from kiosk_gekauft
  group by year
  order by year
""")

provisions: pd.DataFrame = query("""
select
strftime('%Y', "geliefertUm") as year,
sum((verkaufspreis-einkaufspreis)/2) as provisions
from kiosk_gekauft
group by year
order by year
""")

kiosk_donated: pd.DataFrame = query("""
select
strftime('%Y',datum) as year,
sum(betrag) as kiosk_donated
from kiosk_geldtransaktionen
where vonnutzer_id = 3
  and zunutzer_id = 102
GROUP by year
order by year
""")

direct_donated: pd.DataFrame = query("""
select
strftime('%Y',datum) as year,
sum(betrag) as direct_donated
from kiosk_geldtransaktionen
where vonnutzer_id != 3
  and zunutzer_id = 102
GROUP by year
order by year
""")

df = unpaid
df = df.set_index('year')
df = df.join(unpaid_missed_profits.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(paid.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(money_lost.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(worth.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(sell_value.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(buy_value.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(accounts_value.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(buy_value_sold_items.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(sell_value_sold_items.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(provisions.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(kiosk_donated.set_index('year'), on='year', lsuffix='', rsuffix='_r')
df = df.join(direct_donated.set_index('year'), on='year', lsuffix='', rsuffix='_r')

df = df.loc[:, ['unpaid', 'unpaid_missed_profits', 'paid', 'money_lost', 'worth', 'sell_value', 'buy_value', 'accounts_value', 'buy_value_sold_items', 'sell_value_sold_items', 'provisions', 'kiosk_donated', 'direct_donated']]
df = df.fillna(0)

df['vermoegen'] = df.apply(
    lambda x: x.worth + x.sell_value,
    axis=1,
)
df['provision_kiosk'] = df.apply(
    lambda x: (x.sell_value - x.buy_value) / 2.0,
    axis=1,
)

df['theo_income'] = df.apply(
    lambda x: x.paid + x.unpaid,
    axis=1,
)
df['expenses_buyings'] = df.apply(
    lambda x: x.buy_value + x.buy_value_sold_items,
    axis=1,
)
df['provisions_buyings'] = df.apply(
    lambda x: x.provision_kiosk + x.provisions,
    axis=1,
)
df['unpaid_money_loss'] = df.apply(
    lambda x: x.unpaid + x.money_lost,
    axis=1,
)

df['theo_profit'] = df.apply(
    lambda x: x.theo_income - x.expenses_buyings - x.provisions_buyings,
    axis=1,
)
df['real_safe_profit'] = df.apply(
    lambda x: x.theo_profit - x.unpaid_money_loss,
    axis=1,
)
df['real_expected_profit'] = df.apply(
    lambda x: x.real_safe_profit + x.sell_value * x.paid / (x.paid + x.unpaid),
    axis=1,
)
df['bilanz'] = df.apply(
    lambda x: x.real_expected_profit - x.kiosk_donated,
    axis=1,
)

df = df.applymap(lambda x: x/100.0)
df = df.transpose()

df['all'] = df.sum(axis=1)
df.loc['worth', 'all'] = df.loc['worth'].iloc[-2]
df.loc['sell_value', 'all'] = df.loc['sell_value'].iloc[-2]
df.loc['buy_value', 'all'] = df.loc['buy_value'].iloc[-2]
df.loc['accounts_value', 'all'] = df.loc['accounts_value'].iloc[-2]
df.loc['vermoegen', 'all'] = df.loc['vermoegen'].iloc[-2]
df.loc['provision_kiosk', 'all'] = df.loc['provision_kiosk'].iloc[-2]
df.loc['expenses_buyings', 'all'] = df.loc['buy_value', 'all'] + df.loc['buy_value_sold_items', 'all']
df.loc['provisions_buyings', 'all'] = df.loc['provision_kiosk', 'all'] + df.loc['provisions', 'all']
df.loc['theo_profit', 'all'] = df.loc['theo_income', 'all'] - df.loc['expenses_buyings', 'all'] - df.loc['provisions_buyings', 'all']
df.loc['real_safe_profit', 'all'] = df.loc['theo_profit', 'all'] - df.loc['unpaid_money_loss', 'all']
df.loc['real_expected_profit', 'all'] = df.loc['real_safe_profit', 'all'] \
      + df.loc['sell_value', 'all'] * df.loc['paid'].iloc[-2] / (df.loc['paid'].iloc[-2] + df.loc['unpaid'].iloc[-2])
df.loc['bilanz', 'all'] = df.loc['real_expected_profit', 'all'] - df.loc['kiosk_donated', 'all']

print_choices = list(df.columns.values)
df

## Legende

### Tabellenerklärung

| Reihe | Erklärung | "all" |
|------:|:----------|:------|
| **unpaid** | Verkaufswert von Ware, die nicht bezahlt wurde | Summe über alle Jahre |
| **unpaid_missed_profits** | Entgangener Gewinn von Ware, die nicht bezahlt wurde | Summe über alle Jahre |
| **paid** | Bezahlte Ware | Summe über alle Jahre
| **money_lost** | Bargeld-Differenz in Kasse, die vermerkt wurde | Summe über alle Jahre |
| **worth** | Kontostand der Bank. Entspricht dem momentanen theoretischen Gewinn (?)<br> In Kombination mit unbezahlter Ware, Bargeld-Differenz entspricht dies den momentanen realen Profiten (?)<br>Zusätzlich in Kombination mit gespendetem Geld die momentane Bilanz (?) | Wert zum Stichtag |
| **sell_value**<br>**buy_value** | Verkaufswert / Einkaufswert der Waren im Kiosk zum Jahresende bzw. Stichtag der Auswertung | Werte zum Stichtag |
| **accounts_value** | Geldwert auf den Konten aller realen Nutzer:innen zum Jahresende bzw. Stichtag | Wert zum Stichtag |
| **buy_...**<br>**sell_value_sold_items** | Verkaufswert / Einkaufswert aller im Jahr verkauften Waren | Summe über alle Jahre |
| **provisions** | Ausgeschüttete Provisionen für das Erledigen von Einkäufen | Summe über alle Jahre |
| **kiosk_donated** | Spenden aus Profiten des Kiosks heraus | Summe über alle Jahre |
| **direct_donated** | Direktspenden durch Aufrunden-Funktion beim Kauf von Ware | Summe über alle Jahre |
| **vermoegen** | _worth_ (Kontostand der Bank) plus Verkaufswert der Kioskware.<br>Hier fehlt noch unbezahlte Ware und Bargeld-Differenz | Wert zum Stichtag |
| **provision_kiosk** | Bezahlte Provisionen der Kiosk-Ware | Wert zum Stichtag |
| **theo_income** | Geldfluss, der theoretisch der Kiosk-Bank durch Einkäufe zugeführt werden würde, gäbe es keine unbezahlte Ware | Summe über alle Jahre |
| **expenses_buyings** | Ausgegebenes Geld für Einkäufe (gekaufte Ware und Ware im Kiosk) | Summe über alle Jahre |
| **provisions_buyings** | Ausgegebenes Geld für Einkaufsprovisionen (gekaufte Ware und Ware im Kiosk) | Summe über alle Jahre |
| **unpaid_money_loss** | Verkaufswert von unbezahlter Ware und Bargeld-Differenz | Summe über alle Jahre |
| **theo_profit** | Ausgaben für Wareneinkauf und Provisionen gegenüber der theoretischen Bezahlung, falls es keine unbezahlte Ware gäbe.<br> Ohne den Kiosk-Verkaufswert. | Summe über alle Jahre |
| **real_safe_profit** | Theoretischer Profit abzüglich der unbezahlten Ware und der Bargeld-Differenz.<br> Ohne den Kiosk-Verkaufswert. | Summe über alle Jahre |
| **real_expected_profit** | Realer gesicherter Profit plus den Verkaufswert des gesamten Kioskbestandes mit momentaner Unbezahlt-Rate | analog Einzel-Jahresauswertung |
| **bilanz** | Reale erwartete Gewinne abzüglich der Kiosk-Spenden | analog Einzel-Jahresauswertung |

### Notizen

- **Unbezahlte Ware:** ~~Hier nicht den Verkaufswert (EK-Preis + Provision + erwarteter Gewinn) der unbezahlten Ware ausweisen, sondern nur den entgangenen Gewinn! (Plus die Bargeld-Differenz). Erklärung: EK-Preis und Provision sind für die Ware schon in den anderen Posten aufgelistet. Es würde sonst zu Doppelzählung kommen.~~ Doch, alles rechnen, 

In [ ]:
year_choice_waterfall_diagram = print_choices[-1]
year_choice_waterfall_diagram

In [ ]:
print_choices

In [ ]:
_df = df.loc[:, year_choice_waterfall_diagram]

data = []
data.append(dict(
    measure='relative',
    y='Theo. Einnahmen',
    x=_df.theo_income,
))
data.append(dict(
    measure='relative',
    y='Einkauf',
    x=-_df.expenses_buyings,
))
data.append(dict(
    measure='relative',
    y='Einkaufsvergütung',
    x=-_df.provisions_buyings,
))
data.append(dict(
    measure='total',
    y='Theo. Profit<br>(ohne Kioskware)',
    x=None,
))
data.append(dict(
    measure='relative',
    y='Unbezahlt',
    x=-_df.unpaid_money_loss,
))
data.append(dict(
    measure='total',
    y='Realer Profit<br>(ohne Kioskware)',
    x=None,
))
data.append(dict(
    measure='relative',
    y='Prognose Verkauf Kioskware',
    x=_df.real_expected_profit - _df.real_safe_profit,
))
data.append(dict(
    measure='total',
    y='Realer Profit<br>mit verkaufter Kioskware',
    x=None,
))
data.append(dict(
    measure='relative',
    y='Kiosk-Spenden',
    x=-_df.kiosk_donated,
))
data.append(dict(
    measure='total',
    y='Bilanz',
    x=None,
))

fig_waterfall = go.Figure(go.Waterfall(
    name='tst',
    orientation='v',
    connector = {"mode":"between", "line":{"width": 1, "color": "rgba(0, 0, 0, 0.5)", "dash":"solid"}},
    measure=[x['measure'] for x in data],
    x=[x['y'] for x in data],
    y=[x['x'] for x in data],
))

_layout = deepcopy(layout)
_layout['yaxis_zeroline'] = True
_layout['yaxis_zerolinewidth'] = 2
_layout['yaxis_zerolinecolor'] = 'rgba(255,0,0,1)'
_layout['title'] = f'Kioskbetrieb Bilanz "{year_choice_waterfall_diagram}"'
fig_waterfall.update_layout(_layout)
fig_waterfall.show()

## Geldflüsse und Konten

### Funktionsnutzer

Auflistung der nicht normalen Nutzer für weitere Auswertungen

In [ ]:
functional_users = query("""
select username from profil_kioskuser where last_login is null or "aktivBis"='2100-01-01' and is_superuser is false
""")
functional_users

### Geldströme zwischen Nutzergruppen

In [ ]:
functional_transactions = query("""
select
case when vonnutzer_id in (2,3,4,72,73,102)
  then pk.username
  else 'User'
end as von,
case when zunutzer_id in (2,3,4,72,73,102)
  then pk2.username
  else 'User'
end as zu,
sum(betrag) / 100.0 as betrag
from kiosk_geldtransaktionen kg
join profil_kioskuser pk on pk.id = kg.vonnutzer_id
join profil_kioskuser pk2 on pk2.id = kg.zunutzer_id
group by von, zu
""")

functional_transactions

### Kontostände von Nutzergruppen

- Summe aus `Bargeld` und `Bargeld_im_Tresor` bilden den realen Bargeld-Vorrat
- Summe aus `Bank`, `Dieb` und `Bargeld_Dieb` bilden die momentane Bilanz im Kiosk
- Sofern `Spendenkonto` und alle `User`konten geleert sind, haben beide oben genannten Gruppen den gleichen Wert und zeigen den Gewinn im Kiosk

In [ ]:
balances = query("""
select
case when nutzer_id in (2,3,4,72,73,102)
  then pk.username
  else 'User'
end as user,
sum(stand) / 100.0 as stand
from kiosk_kontostand kks
join profil_kioskuser pk on pk.id = kks.nutzer_id
group by user
""")

balances = balances.set_index('user')
balances

In [ ]:
print(
    f' Bargeldbestand:\t\t\t {-round(balances.loc["Bargeld", "stand"] + balances.loc["Bargeld_im_Tresor", "stand"],2)} €',
    '\n',
    f'Eingezahltes Geld auf Nutzerkonten:\t {round(balances.loc["User", "stand"],2)} €',
    '\n',
    f'Spendenkonto:\t\t\t\t {balances.loc["Spendenkonto", "stand"]} €',
    '\n',
    f'Kiosk-Bilanz:\t\t\t\t {round(balances.loc["Bank", "stand"] + balances.loc["Dieb", "stand"] + balances.loc["Bargeld_Dieb", "stand"],2)}€',
)

### Detailanalyse Geldflüsse zur Bank

Hier zeigen sich Geldflüsse zur Kiosk-Bank und von der Kiosk-Bank, die oben nicht berücksichtigt wurden, v.a. Spenden und Konto-Leerungen.
Diese tragen zur positiven Bilanz im Kiosk bei.

Es passt aber noch nicht zusammen...

In [ ]:
bank_money_flows = query("""
with data as (
select datum,
       betrag,
	   kommentar,
	   'User Kauf' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id not in (2,3,4,72,73,102)
  and zunutzer_id = 3
  and kommentar like 'Kauf %'
  
union all

select datum,
       betrag,
	   kommentar,
	   'User Leerung' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id not in (2,3,4,72,73,102)
  and zunutzer_id = 3
  and kommentar like 'Transfer of remaining money %'

union all

select datum,
       betrag,
	   kommentar,
	   'User Spende' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id not in (2,3,4,72,73,102)
  and zunutzer_id = 3
  and kommentar like '%Spende %'

union all

select datum,
       betrag,
	   kommentar,
	   'User Rest' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id not in (2,3,4,72,73,102)
  and zunutzer_id = 3
  and kommentar not like '%Spende %' and kommentar not like 'Transfer of remaining money %' and kommentar not like 'Kauf %'
  
union all

select datum,
       betrag,
	   kommentar,
	   'Dieb Kauf' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id = 4
  and zunutzer_id = 3
  
union all
  
select datum,
       betrag,
	   kommentar,
	   'Bargeld Spende' as zweck
from kiosk_geldtransaktionen
where vonnutzer_id = 2
  and zunutzer_id = 3

union all

select datum,
       -betrag as betrag,
	   kommentar,
	   'Einkauf Erstattung' as zweck
from kiosk_geldtransaktionen
where zunutzer_id not in (2,3,4,72,73,102)
  and vonnutzer_id = 3
  and kommentar like 'Erstattung %'

union all

select datum,
       -betrag as betrag,
	   kommentar,
	   'Rückbuchung' as zweck
from kiosk_geldtransaktionen
where zunutzer_id not in (2,3,4,72,73,102)
  and vonnutzer_id = 3
  and kommentar like 'Rückbuchung %'

union all

select datum,
       -betrag as betrag,
	   kommentar,
	   'Rest' as zweck
from kiosk_geldtransaktionen
where zunutzer_id not in (2,3,4,72,73,102)
  and vonnutzer_id = 3
  and kommentar not like 'Erstattung %' and kommentar not like 'Rückbuchung %'

union all

select datum,
       -betrag as betrag,
	   kommentar,
	   'Dieb Rückbuchung' as zweck
from kiosk_geldtransaktionen
where zunutzer_id = 4
  and vonnutzer_id = 3

union all

select datum,
       -betrag as betrag,
	   kommentar,
	   'Bank Spende' as zweck
from kiosk_geldtransaktionen
where zunutzer_id = 102
  and vonnutzer_id = 3
)

select
strftime('%Y',datum) as year,
zweck,
sum(betrag) / 100.0 as betrag
from data
group by year, zweck
order by year, zweck
""")
bank_money_flows

In [ ]:
from lets_plot import * 
ggplot() + \
geom_bar(aes(x="year", y="betrag", color="zweck", fill="zweck"), data=bank_money_flows, sampling="none" if bank_money_flows.size < 50 else sampling_pick(n=50), stat="identity")